In [ ]:
{-# LANGUAGE BangPatterns, ScopedTypeVariables #-}
import Control.Arrow (Arrow(..))
import Control.Monad
import Control.Monad.Primitive
import Control.Parallel.Strategies
import Control.Concurrent
import Control.Exception (evaluate)

import Data.List
import Data.Ord
import qualified Data.Set as Set
import qualified Data.Vector.Unboxed as U

import Numeric.SpecFunctions
import Numeric.MathFunctions.Constants
import Numeric.MathFunctions.Comparison
import Numeric.Polynomial

import Text.Printf(printf)

import IHaskell.Display
import Graphics.Rendering.Chart.Backend.Cairo
import Graphics.Rendering.Chart.Easy hiding (within)

import Debug.Trace

:l NB/Plot
:l NB/Heatmap

greyColormap 0.2
_heat_map_values (def :: HeatMap Double Double Double)

setNumCapabilities 6

# Incomplete beta

Quick reminder about beta function and (regularized) incomplete beta functions:

Beta function:
$$B(a,b) = \int_0^1 t^{a-1}(1 - t)^{b-1} \,dt $$

Incomplete beta:
$$B(x; a,b) = \int_0^x t^{a-1}(1 - t)^{b-1} \,dt \qquad x \in [0,1]$$

Regularized incomplete beta (from now it'll be referred to simply as incomplete beta)
$$I(x; a,b) = \frac{B(x; a,b)}{B(a,b)}$$


In [ ]:
let fun x = let p  = invIncompleteBeta 7 0.07 x
                x' = incompleteBeta    7 0.07 p
            in x'
toRenderable
  $ plotFunctionsLog [\x -> logBase 10 $ relativeError (fun x) x] (1e-10, 1)

-- Bird's eye view
toRenderable
  $ plotFunctions [invIncompleteBeta 7 0.07] (1e-7,1e-6)
-- Closeup
toRenderable
  $ plotFunctions [invIncompleteBeta 7 0.07] (2.8e-7,3e-7)

Let instrument incomplete beta

In [ ]:
-- | Data type to describe approximation used. See 
data Guess
  = AS109
  | AS64_2
  | NR1
  | NR2
  deriving (Show,Eq)

data Res = Res
  { result :: !Double
  , nIter  :: !Int
  , iters  :: [Double]
  }
  deriving Show 

-- | Compute inverse of regularized incomplete beta function. Uses
-- initial approximation from AS109, AS64 and Halley method to solve
-- equation.
invIncompleteBeta' 
  :: (Double -> Double -> Double -> (Guess,Double)) -- Initial guess
  -> (Double -> Double -> Double -> Double)         -- incomplete beta
  -> Double     -- ^ /p/ > 0
  -> Double     -- ^ /q/ > 0
  -> Double     -- ^ /a/ ∈ [0,0.5] !!!
  -> Res
invIncompleteBeta' oracle calcIbeta p q a
  | p <= 0 || q <= 0 =
      error $ printf "invIncompleteBeta p <= 0 || q <= 0.  p=%g q=%g a=%g" p q a
  | a <  0 || a >  1 =
      error $ printf "invIncompleteBeta x must be in [0,1].  p=%g q=%g a=%g" p q a
  | a == 0 || a == 1 = Res a 0 []
  | a > 0.5          = error "not implemented"
  | otherwise        = invIncompleteBetaWorker oracle calcIbeta (logBeta p q) p q  a


invIncompleteBetaWorker
  :: (Double -> Double -> Double -> (Guess,Double))
  -> (Double -> Double -> Double -> Double)
  -> Double -> Double -> Double -> Double -> Res
-- NOTE: p <= 0.5.
invIncompleteBetaWorker oracle calcIbeta beta a b p = loop (0::Int) [] (snd $ oracle a b p)
  where
    a1 = a - 1
    b1 = b - 1
    -- Solve equation using Halley method
    done i xs x = Res { result = x 
                      , nIter  = i
                      , iters  = reverse $ x : xs
                      }
    loop !i !xs !x
      -- We cannot continue at this point so we simply return `x'
      -- | traceShow (x,f,f',u) False = undefined      
      | isNaN x' = error $ "Got to the NaN a="++show a++" b="++show b++" p="++show p++" x_last="++show x
      
      | x == 0 || x == 1             = done i xs x
      -- When derivative becomes infinite we cannot continue
      -- iterations. It can only happen in vicinity of 0 or 1. It's
      -- hardly possible to get good answer in such circumstances but
      -- `x' is already reasonable.
      | isInfinite f'                = done i xs x
      -- Iterations limit reached. Most of the time solution will
      -- converge to answer because of discreteness of Double. But
      -- solution have good precision already.
      | i >= 100                     = done i xs x
      -- Solution converges      
      | abs dx <= 16 * m_epsilon * x = done i xs x'
      | otherwise                    = loop (i+1) (x:xs) x'
      where
        -- Calculate Halley step.
        f   = calcIbeta a b x  - p
        f'  = exp $ a1 * log x + b1 * log1p (-x) - beta
        u   = f / f'
        -- dx  = u / (1 - 0.5 * min 1 (u * (a1 / x - b1 / (1 - x))))
        dx  = u / (1 - 0.5 * (u * (a1 / x - b1 / (1 - x))))
        -- Next approximation. If Halley step leads us out of [0,1]
        -- range we revert to bisection.
        x'  | z < 0     = x / 2
            | z > 1     = (x + 1) / 2
            | otherwise = z
            where z = x - dx

-- Calculate initial guess. Approximations from AS64, AS109 and
-- Numerical recipes are used.
--
-- Equations are referred to by name of paper and number e.g. [AS64 2]
-- In AS64 papers equations are not numbered so they are refered
-- to by number of appearance starting from definition of
-- incomplete beta.
guessIIBeta :: Double -> Double -> Double -> (Guess,Double)
guessIIBeta a b p
      -- In this region we use approximation from AS109 (Carter
      -- approximation). It's reasonably good (2 iterations on
      -- average)
      | a > 1 && b > 1 =
          let r = (y*y - 3) / 6
              s = 1 / (2*a - 1)
              t = 1 / (2*b - 1)
              h = 2 / (s + t)
              w = y * sqrt(h + r) / h - (t - s) * (r + 5/6 - 2 / (3 * h))
          in (AS109, a / (a + b * exp(2 * w)))
      -- Otherwise we revert to approximation from AS64 derived from
      -- [AS64 2] when it's applicable.
      --
      -- It slightly reduces average number of iterations when `a' and
      -- `b' have different magnitudes.
      | chi2 > 0 && ratio > 1 = (AS64_2, 1 - 2 / (ratio + 1))
      -- If all else fails we use approximation from "Numerical
      -- Recipes". It's very similar to approximations [AS64 4,5] but
      -- it never goes out of [0,1] interval.
      | otherwise = case () of
          _| p < t / w  -> (NR1, (a * p * w) ** (1/a))
           | otherwise  -> (NR2, 1 - (b * (1 - p) * w) ** (1/b))
           where
             lna = log $ a / (a+b)
             lnb = log $ b / (a+b)
             t   = exp( a * lna ) / a
             u   = exp( b * lnb ) / b
             w   = t + u
      where
        -- Formula [2]
        ratio = (4*a + 2*b - 2) / chi2
        -- Quantile of chi-squared distribution. Formula [3].
        chi2 = 2 * b * (1 - t + y * sqrt t) ** 3
          where
            t   = 1 / (9 * b)
        -- `y' is Hasting's approximation of p'th quantile of standard
        -- normal distribution.
        y   = r - ( 2.30753 + 0.27061 * r )
                  / ( 1.0 + ( 0.99229 + 0.04481 * r ) * r )
          where
            r = sqrt $ - 2 * log p

In [ ]:
dropLoop xs
  = map fst
  $ takeWhile (\(x,vals) -> not $ Set.member x vals)
  $ zip xs sets
  where
    sets = scanl (flip Set.insert) Set.empty xs
    
plotIterationsMap oracle aRng bRng n
  = layout_title .~ ("a = " ++ show aRng ++ " b = " ++ show bRng ++ " max=" ++ show (maxN))
  $ layout_plots .~ 
    [ toPlot $ heat_map_values    .~ nIterations
             $ heat_map_color_map .~ blackbodyColormap
             $ def
             ]
  $ def
  where
    extract is | n > 10 = traceShow ("XAB=",x) n
               | otherwise = n
      where (x,n) = maximumBy (comparing snd) is
    maxN = maximum [z | (_,_,z) <- nIterations]
    nIterations = flip using (parList rdeepseq) $ 
      [ ((a1,b1),(a2,b2), fromIntegral (extract iters) :: Double)
      | (a1,a,a2) <- linspaceIntervals aRng n
      , (b1,b,b2) <- linspaceIntervals bRng n
        -- Calculate number of iterations
      , let iters = map (\x -> ((x,a,b), cleverNIter $ invIncompleteBeta' oracle incompleteBeta a b x))
                  $ logspace (1e-10,0.5) 100
      , sqrt (min a b) > abs (a - b)
      ]
    --
    cleverNIter :: Res -> Int
    cleverNIter r 
      | n == 100  = length $ dropLoop $ iters r
      | otherwise = n
      where
        n   = nIter r

In [ ]:
toRenderable $ plotIterationsMap guessIIBeta (100,1000) (100,1000)         200
--toRenderable $ plotIterationsMap guessIIBeta (1000,10000) (1000,10000)     200
--toRenderable $ plotIterationsMap guessIIBeta (10000,100000) (10000,100000) 50
-- toRenderable $ plotIterationsMap (200,300) (500,1000) 40

# New oracle

In [ ]:
temmeGuess a b p
  | a + b >= 5 = case () of
    _| sqrt minA > (maxA-minA)  && minA > 5     -> error "Sec.2"
--     | lambda > 0.2 && lambda < 0.8 && a_b > 10 -> error "Sec. 3 / or power method"
  where 
    --
    minA = min a b
    maxA = max a b
    a_b  = a + b
    
    beta = a - b
    mu   = (a - b) / a


-- | Initial guess from  section 2.
ibetaTemmeGuessSect2 :: Double -> Double -> Double -> Double
ibetaTemmeGuessSect2 a b p
  -- Invert equation 2.1
  | eta2 == 0 = 0.5
  | otherwise = (1 + eta * sqrt((1 - exp(-eta2 / 2)) / eta2)) / 2
  where
    -- Calculate even 
    beta  = b - a
    beta2 = beta  * beta
    beta3 = beta2 * beta
    eta0  = - sqrt(2 / a) * invErfc (2 * p)
    eta   = evaluatePolynomialL (1/a)
      [ eta0
        -- Eq. following 2.15
      , evaluatePolynomialL eta0
        [ -beta * m_sqrt_2 / 2
        , (1 - 2*beta) / 8
        , -(beta * m_sqrt_2) / 48
        , -1 /192
        , beta * m_sqrt_2 / 3840
        ]
        -- Eq. following 2.17
      , evaluatePolynomialL eta0
        [ beta * m_sqrt_2 * (3*beta - 2) / 12
        , (20*beta2 - 12*beta + 1) / 128
        ,  beta * m_sqrt_2 * (20 * beta - 1) / 960
        , (16 * beta2 + 30 * beta - 15) / 4608
        , beta * m_sqrt_2 * (21 * beta + 32) / 53760
        , (-32 * beta2 + 63) / 368640
        , -beta * m_sqrt_2 * (120 * beta + 17) / 25804480
        ]
      , evaluatePolynomialL eta0
        [ beta * m_sqrt_2 * (-75 * beta2 + 80 * beta - 16) / 480
        , (-1080 * beta3 + 868 * beta2 - 90 * beta - 45) / 9216
        , beta * m_sqrt_2 * (-1190 * beta2 + 84 * beta + 373) / 53760
        , (-2240 * beta3 - 2508 * beta2 + 2100 * beta - 165) / 368640
        ]
      ]    
    eta2 = eta * eta;

oracleTemme2 a b p = (AS109, ibetaTemmeGuessSect2 a b p)

In [ ]:
toRenderable $ plotIterationsMap guessIIBeta  (100,1000) (100,1000) 200
toRenderable $ plotIterationsMap oracleTemme2 (100,1000) (100,1000) 200

toRenderable $ plotIterationsMap guessIIBeta  (1000,10000) (1000,10000)     200
toRenderable $ plotIterationsMap oracleTemme2 (1000,10000) (1000,10000)     200

toRenderable $ plotIterationsMap guessIIBeta  (10000,100000) (10000,100000) 50
toRenderable $ plotIterationsMap oracleTemme2 (10000,100000) (10000,100000) 50

In [ ]:
toRenderable $
  let a = 100
      b = 100
      fun x = let p  = invIncompleteBeta a b x
                  x' = incompleteBeta    a b p
              in x'
  in  plotFunctionsLog [\x -> logBase 10 $ relativeError (fun x) x] (1e-10, 1)


In [ ]:
invIncompleteBeta' oracleTemme2 incompleteBeta 6000 6000 0.23

In [ ]:
let a = 7457.5 -- 0.31988259038941824
    b = 7457.5
    is = map (cleverNIter . invIncompleteBeta' oracleTemme2 incompleteBeta a b)
                  $ logspace (1e-10,0.5) 100
    --
    cleverNIter :: Res -> Int
    cleverNIter r 
      | n == 100  = length $ dropLoop $ iters r
      | otherwise = n
      where
        n   = nIter r
 in is

In [ ]:
invIncompleteBeta' oracleTemme2 incompleteBeta 7457.5 7457.5 0.31988259038941824

In [ ]:
toRenderable $
  let a = 7457.5
  in ulpPlot [incompleteBeta a a] (addUlps (-1000) 0.5) 2000

In [ ]:
gogogo (x,a,b) = toRenderable $  
  ulpPlot [incompleteBeta a b] (addUlps (-n) p) (2*n)
  where
    p = invIncompleteBeta a b x
    n = 1000
-- gogogo (x,a,b) = invIncompleteBeta' guessIIBeta incompleteBeta a b x
                                                                                        
                                              

In [ ]:
gogogo (0.39992661226118575,813.25,817.75)
gogogo (0.2046497432684883,831.25,808.75)                                                                                           
gogogo (0.1636897570509756,853.75,831.25)                                                                                           
gogogo (0.2046497432684883,966.25,984.25)  